<a href="https://colab.research.google.com/github/alinastepd/application/blob/main/Twitter%20Sentiment%20Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Dataset 1 (Tweets.csv).

In [33]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer
import string

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [34]:
dataset1 = pd.read_csv('Tweets.csv')
dataset1.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [35]:
X = dataset1['text']
Y = dataset1['airline_sentiment']

Cleaning the text data

In [36]:
stop_words = stopwords.words('english')
punct = string.punctuation
stemmer = PorterStemmer()

In [37]:
cleaned_data = []
for i in range(len(X)):
  dataset1 = re.sub('[a-zA-Z]', ' ', X.iloc[i])
  dataset1 = dataset1.lower().split()
  dataset1 = [stemmer.stem(word) for word in dataset1 if (word not in stop_words) and (word not in punct)]
  dataset1 = ' '.join(dataset1)
  cleaned_data.append(dataset1)


In [38]:
Y

,airline_sentiment
0,neutral
1,positive
2,neutral
3,negative
4,negative
...,...
14635,positive
14636,negative
14637,neutral
14638,negative


In [39]:
sentiments = ['negative', 'neutral', 'positive'] #negative - 0, neutral - 1, positive - 2
Y = Y.apply(lambda X: sentiments.index(X))

In [31]:
Y.head()

,airline_sentiment
0,1
1,2
2,1
3,0
4,0


In [41]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 3000, stop_words = ['virginamerica', 'unit'])
X_fin = cv.fit_transform(cleaned_data).toarray()

In [42]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
model = MultinomialNB()

In [43]:
X_train, X_test, Y_train, Y_test = train_test_split(X_fin, Y, test_size = 0.3)

In [44]:
model.fit(X_train, Y_train)

MultinomialNB()

In [45]:
Y_pred = model.predict(X_test)

In [46]:
from sklearn.metrics import classification_report
cf = classification_report(Y_test, Y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.64      0.99      0.77      2769
           1       0.49      0.03      0.05       940
           2       0.16      0.01      0.02       683

    accuracy                           0.63      4392
   macro avg       0.43      0.34      0.28      4392
weighted avg       0.53      0.63      0.50      4392



Dataset 2